In [1]:
import numpy as np
from pedalboard import Pedalboard, Chorus, Reverb, Mix, Chain, Gain, Plugin
from pedalboard.io import AudioFile
from plgUtil import *
from paramOptim import optimize_plg_params
from faustify import faustify
import typing
from random import choice, randint
from genAlgo import *

In [2]:
a = Pedalboard([Gain()])

In [2]:
a = Pedalboard([Gain()])
for _ in range(6) :
    mutate(a)

print(faustify(a))



ACC: ((GAIN)), ((REVERB)):> _, 
     ACC[-2]: _
ACC: ((GAIN)), ((((GAIN)), ((REVERB)):> _)):> _, 
     ACC[-2]: _
(CHORUS) : (CHORUS) : (CHORUS) : (CHORUS) <: ((GAIN)), ((((GAIN)), ((REVERB)):> _)):> _


In [3]:
a


<Pedalboard with 3 plugins: [<pedalboard.Mix with 2 plugins: [<pedalboard.Chain with 2 plugins: [<pedalboard.Chain with 1 plugin: [<pedalboard.Gain gain_db=1 at 0x5652a89e8660>] at 0x5652a8903650>, <pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x5652a8ac3330>] at 0x5652a8a54250>, <pedalboard.Chain with 3 plugins: [<pedalboard.Chain with 1 plugin: [<pedalboard.Gain gain_db=1 at 0x5652a8a837c0>] at 0x5652a893bca0>, <pedalboard.Chain with 1 plugin: [<pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x5652a8ac43f0>] at 0x5652a8955960>, <pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x5652a8ac3a40>] at 0x5652a898e660>] at 0x5652a8943be0>, <pedalboard.Chain with 1 plugin: [<pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x5652a8ab5850>] at 0x5652a88616f0>, <pedalboard.Gain gain_db=1 at 0x5652a8a32860>]>

In [75]:
print(a[1])
print(a[2])
a[3]

<pedalboard.Chain with 1 plugin: [<pedalboard.Reverb room_size=0.5 damping=0.5 wet_level=0.33 dry_level=0.4 width=1 freeze_mode=0 at 0x56034d51b3f0>] at 0x56034d4f92a0>
<pedalboard.Chain with 1 plugin: [<pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x56034d608090>] at 0x56034d62c1d0>


<pedalboard.Mix with 2 plugins: [<pedalboard.Chain with 1 plugin: [<pedalboard.Gain gain_db=1 at 0x56034d5eec50>] at 0x56034d3a6220>, <pedalboard.Chain with 1 plugin: [<pedalboard.Gain gain_db=1 at 0x56034d5ec940>] at 0x56034d4f9200>] at 0x56034d4eab50>

In [76]:
simplify_board(a)

In [78]:
a[1]

<pedalboard.Chain with 2 plugins: [<pedalboard.Reverb room_size=0.5 damping=0.5 wet_level=0.33 dry_level=0.4 width=1 freeze_mode=0 at 0x56034d51b3f0>, <pedalboard.Chorus rate_hz=1 depth=0.25 centre_delay_ms=7 feedback=0 mix=0.5 at 0x56034d608090>] at 0x56034d609d70>

In [65]:
simplify_chain(a[0][0][0])
a[0][1]

<pedalboard.Chain with 1 plugin: [<pedalboard.Gain gain_db=1 at 0x56034d4f5a70>] at 0x56034d35cc80>

In [45]:
dir(Reverb())

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'damping',
 'dry_level',
 'freeze_mode',
 'process',
 'reset',
 'room_size',
 'wet_level',
 'width']

In [ ]:
def board_fitness(board, plg, input_buf, target_buf, samplerate, tol=1.0e-5) :
    return

In [38]:
a = Pedalboard([Reverb()])
b = Pedalboard([Mix([Chorus(), a])])
print(faustify(b))
a.append(Chorus())
print(faustify(b))
help(Pedalboard.remove)
a.insert(0, Chorus())
print(faustify(b))
dir(a)

(Node), ((Node)) :> _
(Node), ((Node : Node)) :> _
Help on instancemethod in module pedalboard_native:

remove(...)
    remove(self: pedalboard_native.PluginContainer, arg0: pedalboard_native.Plugin) -> None

(Node), ((Node : Node : Node)) :> _


['__call__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'insert',
 'process',
 'remove',
 'reset']

In [84]:
board = Pedalboard([Pedalboard([])])
plg = list(board[0])
board.append(Chain(plg))
board


<Pedalboard with 2 plugins: [<pedalboard.Chain with 0 plugins: [] at 0x56210c118b80>, <pedalboard.Chain with 0 plugins: [] at 0x56210be0c9b0>]>

In [86]:
Reverb() + Chain()

TypeError: unsupported operand type(s) for +: 'pedalboard_native.Reverb' and 'pedalboard_native.utils.Chain'

In [68]:
Pedalboard

False

### Points to consider
- can insert